In [135]:
import pandas as pd

df_backstreetboys = pd.read_csv('Dados\Backstreet_Boys_Lyrics_score.csv') 
df_backstreetboys = df_backstreetboys.drop(columns=["Unnamed: 0", "Unnamed: 0.1"]) # Remove colunas desnecessárias

df_arcticmonkeys = pd.read_csv('Dados\lyrics.csv')

In [169]:
# Divide o dataset em 2, um para cada banda
df_bb_treino = df_backstreetboys.iloc[:50] # Seleciona as primeiras linhas do dataset
df_bb_teste = df_backstreetboys.iloc[50:100] # Seleciona as últimas linhas do dataset

df_am_treino = df_arcticmonkeys.iloc[:50] # Seleciona as primeiras linhas do dataset
df_am_teste = df_arcticmonkeys.iloc[50:100] # Seleciona as últimas linhas do dataset


In [161]:
# Probabilidade de ser de cada banda

p_am = len(df_am_treino) / (len(df_am_treino) + len(df_bb_treino))
p_bb = len(df_bb_treino) / (len(df_am_treino) + len(df_bb_treino))

0.5


In [174]:

palavras_inuteis = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o', 'and', 'i', 'or', 'and','me','he','him','himself','she','her','herself','we','our','ourself', 'the','they','them','themselves','to', 'you','your','yours','yourself','yourselves','it',
'its','itself','what','which','who','whom','this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','will','would','should','can','could','ought','im','youre','hes','shes','its','were','theyre','ive','youve','weve','theyve','id','youd','hed','shed','wed','theyd','ill','youll','hell','shell','well','theyll','isnt','arent','wasnt','werent','hasnt','havent','hadnt','doesnt','dont','didnt','wont','wouldnt','shant','shouldnt','cant','cannot','couldnt','mustnt','lets','thats','whos','whats','heres','theres','whens','wheres','whys','hows','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','upon','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both',
'each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','can','will','just','don','should','now','1','2','3','4','5','6','7','8','9','0']

def separador_palavra(text):
    return text.split() # Separa o texto em palavras a partir dos espaços

def limpar_musica(musica_am):
    musica_am = musica_am.lower()
    caracteres_a_remover = [',', '.', '?', '!', '(', ')', ';', ':', '"', "'", '-', '[', ']', '\r\n', 'verse', 'chorus']

    for caractere in caracteres_a_remover:
        musica_am = musica_am.replace(caractere, ' ')

    return musica_am

# def apagar_palavras_inuteis(palavras_inuteis, palavras):

# for palavra in palavras_inuteis:
#     if palavra in palavras:
#         palavras.remove(palavra)

        
# Probabilidade de cada palavra aparecer em Arctic Monkeys
dict_am = {}
dict_palavras_am = {}

for musica_am in df_am_treino['lyrics']:
    musica_am = limpar_musica(musica_am) # Aplica a função limpar_musica a cada linha da coluna lyrics
    dict_am['palavras'] = separador_palavra(musica_am) # Aplica a função separador_palavra a cada linha da coluna lyrics
    df_am_novo = pd.DataFrame(dict_am)
    palavras_am = df_am_novo['palavras'].explode() # Transforma a coluna palavras em um novo df para calcular a frequência de cada palavra corretamente

    palavras_unicas_am = palavras_am.unique() # Remove palavras repetidas

    for palavra_am in palavras_unicas_am:
        if palavra_am not in dict_palavras_am:
            dict_palavras_am[palavra_am] = 1
        else:
            dict_palavras_am[palavra_am] += 1

dict_prob_am = {}

for palavra_am in dict_palavras_am:
    dict_prob_am[palavra_am] = dict_palavras_am[palavra_am] / len(df_am_treino)

print(dict_am)

{'palavras': ['1', 'so', 'you', 're', 'on', 'the', 'prowl', 'wondering', 'whether', 'she', 'left', 'already', 'or', 'not', 'leather', 'jacket', 'collar', 'popped', 'like', 'antenna', 'never', 'knowing', 'when', 'to', 'stop', 'sunglasses', 'indoors', 'par', 'for', 'the', 'course', 'lights', 'in', 'the', 'floors', 'and', 'sweat', 'on', 'the', 'walls', 'cages', 'and', 'poles', 'call', 'off', 'the', 'search', 'for', 'your', 'soul', 'or', 'put', 'it', 'on', 'hold', 'again', 'she', 's', 'having', 'a', 'sly', 'indoor', 'smoke', 'and', 'she', 'calls', 'the', 'folks', 'who', 'run', 'this', 'her', 'oldest', 'friends', 'sipping', 'a', 'drink', 'and', 'laughing', 'at', 'imaginary', 'jokes', 'as', 'all', 'the', 'signals', 'are', 'sent', 'her', 'eyes', 'invite', 'you', 'to', 'approach', 'and', 'it', 'seems', 'as', 'though', 'those', 'lumps', 'in', 'your', 'throat', 'that', 'you', 'just', 'swallowed', 'have', 'got', 'you', 'going', 'come', 'on', 'come', 'on', 'come', 'on', 'come', 'on', 'come', 'on',

In [139]:
# Probabilidade de cada palavra aparecer em Backstreets Boys
dict_bb = {}
dict_palavras_bb = {}

for musica_bb in df_bb_treino['Lyrics']:
    musica_bb = musica_bb.lower().replace(',', '').replace('.', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').replace(';', '').replace(':', '').replace('"', '').replace("'", '').replace('-', '').replace('[', '').replace(']', '').replace('\r\n', ' ')
    dict_bb['palavras'] = separador_palavra(musica_bb) # Aplica a função separador_palavra a cada linha da coluna lyrics
    df_bb_novo = pd.DataFrame(dict_bb)
    palavras_bb = df_bb_novo['palavras'].explode() # Transforma a coluna palavras em um novo df para calcular a frequência de cada palavra corretamente

    palavras_unicas_bb = palavras_bb.unique() # Remove palavras repetidas

    for palavra_bb in palavras_unicas_bb:
        if palavra_bb not in dict_palavras_bb:
            dict_palavras_bb[palavra_bb] = 1
        else:
            dict_palavras_bb[palavra_bb] += 1

dict_prob_bb = {}

for palavra_bb in dict_palavras_bb:
    dict_prob_bb[palavra_bb] = dict_palavras_bb[palavra_bb] / len(df_bb_treino)

In [140]:
# Probabilidade de cada palavra aparecer em ambas as bandas
dict_palavras_both = {}

for palavra_am_new in dict_palavras_am:
    dict_palavras_both[palavra_am_new] = dict_palavras_am[palavra_am_new]

for palavra_bb_new in dict_palavras_bb:
    if palavra_bb_new not in dict_palavras_both:
        dict_palavras_both[palavra_bb_new] = dict_palavras_bb[palavra_bb_new]
    else:
        dict_palavras_both[palavra_bb_new] += dict_palavras_bb[palavra_bb_new]

dict_prob_both = {}

for palavra_both in dict_palavras_both:
    dict_prob_both[palavra_both] = dict_palavras_both[palavra_both] / (len(df_am_treino) + len(df_bb_treino))

In [141]:
# Probabilidade de ser do Arctic Monkeys dado que a palavra aparece
# Teorema de Bayes 
# P(A|B) = P(B|A) * P(A) / P(B)
dict_prob_am_dado_palavra = {}

for palavra in dict_prob_am:
    dict_prob_am_dado_palavra[palavra] = (dict_prob_am[palavra] * p_am) / dict_prob_both[palavra]

In [142]:
# def classificador(dict_am, dict_bb, dict_both):
#     certo = 0
#     errado = 0

#     for palavra in dict_both:
#         if palavra not in dict_am or palavra not in dict_bb:
#             continue
#         if dict_am[palavra] > dict_bb[palavra]:
#             if dict_prob_am_dado_palavra[palavra] > 0.5:
#                 certo += 1
#             else:
#                 errado += 1
#         else:
#             if dict_prob_am_dado_palavra[palavra] < 0.5:
#                 certo += 1
#             else:
#                 errado += 1
    
#     return certo, errado

# certo, errado = classificador(dict_prob_am, dict_prob_bb, dict_prob_both)

# prob_certo = certo / (certo + errado)

# prob_certo

def classficador(dict_am, dict_bb, dict_both):
    contador_am = 0
    contador_bb = 0
    contador_both = 0

    for palavra in dict_both:
        if palavra not in dict_am or palavra not in dict_bb:
            continue
        if dict_am[palavra] > dict_bb[palavra]:
            contador_am += 1
        if dict_am[palavra] == dict_bb[palavra]:
            contador_both += 1
        else:
            contador_bb += 1

    return contador_am, contador_bb, contador_both

contador_am, contador_bb, contador_both = classficador(dict_prob_am, dict_prob_bb, dict_prob_both)

contador_am, contador_bb, contador_both

(137, 359, 111)